# Cross-validated MANOVA in Python
I've worked quite a bit on 'decoding (machine-learning) analyses' of fMRI data. A while back, I came across the article by [Allefeld & Haynes (2014)](http://www.sciencedirect.com/science/article/pii/S1053811913011920) on 'cross-validated MANOVA', an multivariate encoding-type of analysis. In this notebook, I've tried to implement this analysis and annotated some of the steps. 

# MGLM
The MANOVA is a test derived from the multivariate general linear model (MGLM). Conceptually, a MANOVA tests how much variance in a set of target variables (dependent variables; $\mathbf{y}$) by a (set of) predictor(s) ($\mathbf{X}$); in other words, it's a "multivariate analysis of variance", as the name implies. In my understanding (which is often more conceptual than mathematical), the first step in the analysis described by Allefeld & Haynes is to simply calculate the parameters of the GLM ($\beta$) for all target variables. The GLM, here, is defined as:

\begin{align}
y = \beta\mathbf{X} + \epsilon
\end{align}

in which $\beta$ represents the GLM's parameter(s) and $\epsilon$ the model's errors. Now, the parameter(s) $\beta$ can be found by the GLM's analytical solution:

\begin{align}
\hat{\beta} = (\mathbf{X}'\mathbf{X})^{-1}y
\end{align}

In which $X$ represents the design-matrix (predictors), a $N\ (samples) \times P\ (predictors)$ matrix, and $y$ represents a $N \times 1$ column-vector with the target variable. I recently found at that you can estimate the parameters for different target variables (so different $y$ variables) corresponding to the model:

\begin{align}
\mathbf{y} = \beta\mathbf{X} + \Xi
\end{align}

in which $\mathbf{y}$ is now a $N \times K$ (number of target variables) matrix and $\Xi$ a $N \times K$ matrix with the model errors. As such, the parameters of this model can be found by vectorizing the previously outlined formula for the (univariate) GLM as follows:

\begin{align}
\hat{\mathbf{\beta}} = (\mathbf{X}'\mathbf{X})^{-1}\mathbf{y}
\end{align}

in which, now, $\mathbf{y}$ is an $N \times K$ (target-variables) matrix and $\beta$ is an $P \times K$ matrix.

Let's check out an example. We'll use the 'canonical' Iris-dataset, which contains data of three types of Iris flowers (let's say, class C1, C2, and C3) associated with four types of variables (let's call them V1, V2, V3, and V4). In the context of a MANOVA, we could investigate whether the factor 'flower type' has a multivariate effect on the four measured variables. Thus, here flower type represents the design-matrix ($\mathbf{X}$) and the measured variables represent the target variables ($\mathbf{y}$). Note, this may seem counterintuitive for people that are familiar with this dataset in the context of machine learning, in which it is used to predict flower type ($y$) as a function of the flower features ($\mathbf{X}$). This is, essentially, just a manifestation of a different question about the data. In neuroimaging, however, this difference is quite important (check out the awesome article by [Naselaris and colleagues](http://www.sciencedirect.com/science/article/pii/S1053811910010657) on this topic).

Anyway, let's check out how the parameters are calculated in the MGLM. We'll first load the data:

In [1]:
from sklearn.datasets import load_iris
import numpy as np

y, flower_types = load_iris(return_X_y=True)
print("Shape of y (flower properties): %s" % (y.shape,))
print("Shape of flower types variable: %s" % (flower_types.shape,), end='\n\n')
print(flower_types)

Shape of y (flower properties): (150, 4)
Shape of flower types variable: (150,)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


To investigate the effect of flower type on the target variables in the context of the GLM, we need to create a separate predictor for each class (C1, C2, C3), which contains all zeros except for the samples that correspond to that particular class. This process is known (albeit in a different context) as 'one-hot encoding'. Let's do this:

In [2]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
X = ohe.fit_transform(flower_types[:, np.newaxis]).astype(float)
print("Shape of X (flower type): %s" % (X.shape,))
print(X)

Shape of X (flower type): (150, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.

Sweet! Exactly what we wanted. Now, let's solve the parameters of this GLM-model:

In [3]:
betas = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)
print("Shape of betas: %s" % (betas.shape,))
print(betas)

Shape of betas: (3, 4)
[[ 5.006  3.418  1.464  0.244]
 [ 5.936  2.77   4.26   1.326]
 [ 6.588  2.974  5.552  2.026]]


In fact, the betas (given this design-matrix) represent the mean of each class (rows) for each variable (columns).

## The MANOVA
Now, the Allefeld & Haynes paper define a 'pattern-distinctness' statistic $\hat{D}$, which is based on the Barlett-Lawley-Hotelling Trace $T_{BLH}$, which represents the trace of the product of the between-class covariance (I call it $B$) and the inverse of the within-class covariance (they call it $\Sigma$). Thus, the formula for the Barlett-Lawley-Hotelling trace become:

\begin{align}
T_{BLH} = \mathrm{trace}(B\Sigma^{-1})
\end{align}

In the paper, $B$ is calculated using the following formula (this becomes a bit hairy):

\begin{align}
B = \hat{\beta}_{\Delta}'\mathbf{X}'\mathbf{X}\hat{\beta}_{\Delta}
\end{align}

in which $\hat{\beta}_{\Delta}$ is calculated using the contrast-matrix $C$ as follows:

\begin{align}
\hat{\beta}_{\Delta} = (C'C)^{-1}C\hat{\beta}
\end{align}

Fortunately, $\Sigma$ is simply the inner product of the residuals from the model:

\begin{align}
\Sigma = (\mathbf{y} - \hat{\beta}\mathbf{X})'(\mathbf{y} - \hat{\beta}\mathbf{X}) = \hat{\Xi}'\hat{\Xi}
\end{align}

Alright, but what contrast should we use? Well, that of course depends on your question. In our case, we said we were interested in the effect of flower type on the target variables. Given that we have three flower types (C1, C2, C3), an F-test of $K - 1$ pairwise difference contrasts (e.g. C1 - C2 and C1 - C3) should do the trick. As such, we could define $C$ as:

\begin{align}
C =
    \begin{bmatrix}
    1 & -1 & 0\\1 & 0 & -1
    \end{bmatrix}
\end{align}

Let's define it below

In [4]:
C = np.array([
    [1, 0, -1],
    [0, 1, -1]    
])

Now, let's calculate $\hat{\beta}_{\Delta}$ ...

In [5]:
beta_delta = C.T.dot(np.linalg.pinv(C.T.dot(C)).dot(C.T).T).dot(betas)
print("Shape B-delta: %s" % (beta_delta.shape,))

Shape B-delta: (3, 4)


... and $B$:

In [6]:
B = beta_delta.T.dot(X.T.dot(X)).dot(beta_delta)

... $\Sigma$:

In [7]:
sigma = (y - X.dot(betas)).T.dot((y - X.dot(betas)))

... and finally $T_{BLH}$!

In [8]:
T_blh = np.trace(B.dot(np.linalg.inv(sigma)))
print(T_blh)

32.5495246636


I always check whether I got the right result by checking it against some existing implementation. Fortunately, there is a MANOVA implementation in the master branch (not yet op PyPI) of the statsmodels package. So, let's check whether it matches:

In [9]:
from statsmodels.multivariate.manova import MANOVA

mglm = MANOVA(endog=y, exog=X)
fitted_model = mglm.mv_test([    
    ('One-way MANOVA', C)
])

fitted_model.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Multivariate linear model
================================================================
                                                                
----------------------------------------------------------------
     One-way MANOVA      Value  Num DF  Den DF   F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda  0.0235 8.0000 288.0000  198.7110 0.0000
         Pillai's trace  1.1872 8.0000 290.0000   52.9486 0.0000
 Hotelling-Lawley trace 32.5495 8.0000 203.4024  583.4914 0.0000
    Roy's greatest root 32.2720 4.0000 145.0000 1169.8585 0.0000
================================================================

"""

Nice! Exactly the same (the value in statsmodels is called *Hotelling-Layley trace*).

## The *cross-validated* MANOVA
Importantly, the paper suggests to cross-validate the MANOVA procedure (and thus the calculation of the statistic of interest, $\hat{D}$), because $\hat{D}$ is a distance-measure -- essentially an extension of the mahalanobis distance for more than two groups (levels) -- which are inherently positively biased because distances cannot be negative. Cross-validated makes sure the estimate becomes unbiased. 

In [17]:
from sklearn.model_selection import StratifiedKFold

M = 5
cv = StratifiedKFold(n_splits=5)
T_blh_cv = np.zeros(5)

for i, (train_idx, test_idx) in enumerate(cv.split(X=y, y=flower_types)):
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    betas_train = np.linalg.pinv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)
    betas_test = np.linalg.pinv(X_test.T.dot(X_test)).dot(X_test.T).dot(y_test)
    beta_delta_train = C.T.dot(np.linalg.pinv(C.T.dot(C)).dot(C.T).T).dot(betas_train)
    beta_delta_test = C.T.dot(np.linalg.pinv(C.T.dot(C)).dot(C.T).T).dot(betas_test)
    B_cv = beta_delta_train.T.dot(X_test.T.dot(X_test)).dot(beta_delta_test)
    sigma_cv = (y_train - X_train.dot(betas_train)).T.dot((y_train - X_train.dot(betas_train)))
    T_blh_cv[i] = np.trace(B_cv.dot(np.linalg.inv(sigma_cv)))

print(T_blh_cv.mean())
N = X.shape[0]
((M - 1)*(N - np.linalg.matrix_rank(X))-y.shape[1]-1) / ((M - 1) * N)

8.26461949482


0.97166666666666668

Hmm, this is much lower than the uncrossvalidated statistic. Need to check up on this.